<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-16 11:01:14
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 64.10 K (0.46%)
Current PnL: -21.38 L (-14.24%)
CY Booked + Current PnL: -9.48 L (-6.31%)
-------------------
Total profit:  1.72 L
Total loss:  -23.10 L
-------------------
Total Booked + Current PnL: 17.50 L (14.22%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.49%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.57 L (63.21%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.44,59.0,X-LC,3.79,214980.0,10480.0,12813.0,0.67,5.12,5.96,11.39,10.0,0.82,1.55,14.54,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,95.94,48.0,M-SC,1.67,84430.0,-16347.0,16439.0,0.16,-16.22,19.47,0.09,245.0,-0.99,0.61,10.92,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.02,58.0,X-MC,1.26,212040.0,20298.0,17451.0,0.11,10.59,8.23,19.69,99.0,1.16,1.53,18.10,XY25,NTT,AC
33,ICICIGI,2260.25,-12.91,68.0,X-MC,2.10,149628.0,13495.0,17626.0,0.08,9.91,11.78,22.86,91.0,0.77,1.08,23.78,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-14.04,49.0,X-LC,2.85,214968.0,4602.0,24184.0,0.27,2.19,11.25,13.68,37.0,0.19,1.55,19.07,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SONACOMS,1006.0,-33.78,69.0,M-SC,12.26,82478.0,-18681.0,93571.0,4.21,-18.47,113.45,74.03,202.0,-0.20,0.59,16.30,AR,BTT,AUTO
84,WHIRLPOOL,2270.0,-45.34,50.0,M-SC,4.25,92415.0,916.0,77832.0,4.16,1.00,84.22,86.07,223.0,0.01,0.66,34.20,XR,NTT,DURABLES
63,SHALBY,327.0,1195.06,57.0,M-SC,23.05,176161.0,-4982.0,50453.0,3.82,-2.75,28.64,25.10,235.0,-0.10,1.27,41.18,XY24,NTT,HEALTHCARE
24,EASEMYTRIP,26.4,-17.82,43.0,M-SC,25.54,79040.0,-97775.0,178575.0,3.18,-55.30,225.93,45.70,197.0,-0.55,0.57,3.18,XY24,NTT,TRAVEL
4,ANGELONE,3033.0,16.84,71.0,X-SC,9.31,206451.0,15445.0,42261.0,2.96,8.09,20.47,30.21,157.0,0.37,1.49,30.66,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HATHWAY,31.40,856.52,36.0,H-SC,5.20,82260.0,-26460.0,106140.0,-1.79,-24.34,129.03,73.29,161.0,-0.25,0.59,10.03,XR,NTT,ENTERTAINMENT
15,CAMPUS,393.00,-23.18,57.0,M-SC,2.78,160003.0,-7772.0,63617.0,-1.25,-4.63,39.76,33.28,210.0,-0.12,1.15,24.01,XY24,NTT,FOOTWEAR
10,BANDHANBNK,400.00,-13.99,45.0,H-SC,2.90,215954.0,-62605.0,320044.0,-1.25,-22.47,148.20,92.42,151.0,-0.20,1.55,24.80,XY24,NTT,BANKS
50,PGHH,17907.65,-26.84,57.0,X-MC,3.25,211905.0,11085.0,56706.0,-1.06,5.52,26.76,33.76,80.0,0.20,1.52,11.00,X40,ATH,FMCG
55,RELAXO,1176.00,-39.59,43.0,X-SC,4.02,82344.0,-62816.0,138741.0,-0.90,-43.27,168.49,52.31,136.0,-0.45,0.59,10.22,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-186.36,69.0,M-SC,17.60,149256.0,36.0,144151.0,0.52,0.02,96.58,96.62,208.0,0.00,1.07,61.53,XY25,NTT,FINANCE
59,SAIL,228.0,43.54,51.0,M-MC,12.24,227115.0,2153.0,165499.0,1.23,0.96,72.87,74.53,192.0,0.01,1.63,33.52,XY24,BTT,STEEL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,61.0,H-SC,11.92,131733.0,-10746.0,30272.0,0.45,-7.54,22.98,13.70,163.0,-0.35,0.95,55.48,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,95.94,48.0,M-SC,1.67,84430.0,-16347.0,16439.0,0.16,-16.22,19.47,0.09,245.0,-0.99,0.61,10.92,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,51.0,H-SC,1.93,222012.0,-47655.0,83987.0,0.42,-17.67,37.83,13.47,138.0,-0.57,1.60,12.89,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,44.0,H-SC,2.19,139932.0,-35971.0,77998.0,0.51,-20.45,55.74,23.89,149.0,-0.46,1.01,20.85,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.05,46.0,H-MC,7.14,102657.0,-29178.0,71398.0,0.66,-22.13,69.55,32.03,98.0,-0.41,0.74,14.98,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,55.0,M-LC,6.13,208532.0,5362.0,36764.0,0.89,2.64,17.63,20.74,55.0,0.15,1.50,8.33,XY25,NTT,FINANCE
59,SAIL,228.00,43.54,51.0,M-MC,12.24,227115.0,2153.0,165499.0,1.23,0.96,72.87,74.53,192.0,0.01,1.63,33.52,XY24,BTT,STEEL
31,HINDZINC,730.22,36.19,67.0,M-LC,9.50,222002.0,16926.0,95638.0,-0.56,8.25,43.08,54.89,52.0,0.18,1.60,34.34,X5K,ATH,METALS
85,WIPRO,420.00,-11.52,56.0,M-LC,5.76,155130.0,4185.0,105271.0,0.72,2.77,67.86,72.51,53.0,0.04,1.12,8.96,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,54.0,M-LC,8.73,157679.0,16839.0,27657.0,-0.55,11.96,17.54,31.59,66.0,0.61,1.13,41.61,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.19,67.0,M-LC,9.50,222002.0,16926.0,95638.0,-0.56,8.25,43.08,54.89,52.0,0.18,1.60,34.34,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,60.0,H-MC,13.01,193142.0,13334.0,99082.0,-0.60,7.42,51.30,62.52,88.0,0.13,1.39,44.03,XR,NTT,BANKS
85,WIPRO,420.00,-11.52,56.0,M-LC,5.76,155130.0,4185.0,105271.0,0.72,2.77,67.86,72.51,53.0,0.04,1.12,8.96,XR,NTT,IT
84,WHIRLPOOL,2270.00,-45.34,50.0,M-SC,4.25,92415.0,916.0,77832.0,4.16,1.00,84.22,86.07,223.0,0.01,0.66,34.20,XR,NTT,DURABLES
36,INDIAMART,4810.62,-55.39,40.0,H-SC,8.07,124704.0,1368.0,130253.0,1.00,1.11,104.45,106.72,119.0,0.01,0.90,24.75,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,5.40,145127.0,-131396.0,241462.0,2.29,-47.52,166.38,39.81,54.0,-0.54,1.04,2.29,XY24,NTT,AUTO
28,HAVELLS,2069.16,-8.45,33.0,X-MC,1.89,227042.0,-20689.0,95744.0,-0.60,-8.35,42.17,30.30,92.0,-0.22,1.63,3.83,X40,ATH,ELECTRICAL
51,QUESS,986.00,-47.11,35.0,X-SC,38.51,52297.0,-12709.0,163637.0,0.21,-19.55,312.90,232.18,198.0,-0.08,0.38,1.99,XY24,NTT,MISC
12,BATAINDIA,2096.00,-36.44,40.0,X-SC,4.33,94935.0,-33735.0,79033.0,1.79,-26.22,83.25,35.20,219.0,-0.43,0.68,8.96,X40,NTT,FOOTWEAR
34,ICICIPRULI,790.00,-22.94,40.0,X-MC,2.20,132662.0,-3126.0,45875.0,-0.17,-2.30,34.58,31.48,107.0,-0.07,0.95,9.54,X40,ATH,INSURANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.83,45.0,H-LC,0.62,156105.0,-29990.0,77475.0,0.11,-16.12,49.63,25.51,15.0,-0.39,1.12,15.04,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.02,58.0,X-MC,1.26,212040.0,20298.0,17451.0,0.11,10.59,8.23,19.69,99.0,1.16,1.53,18.10,XY25,NTT,AC
28,HAVELLS,2069.16,-8.45,33.0,X-MC,1.89,227042.0,-20689.0,95744.0,-0.60,-8.35,42.17,30.30,92.0,-0.22,1.63,3.83,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-12.91,68.0,X-MC,2.10,149628.0,13495.0,17626.0,0.08,9.91,11.78,22.86,91.0,0.77,1.08,23.78,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-7.20,62.0,X-LC,2.37,285620.0,20194.0,36588.0,-0.07,7.61,12.81,21.40,11.0,0.55,2.05,14.76,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,35.0,X-SC,38.51,52297.0,-12709.0,163637.0,0.21,-19.55,312.90,232.18,198.0,-0.08,0.38,1.99,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.83,14.0,X-LC,5.40,145127.0,-131396.0,241462.0,2.29,-47.52,166.38,39.81,54.0,-0.54,1.04,2.29,XY24,NTT,AUTO
73,TCS,4389.97,-30.29,44.0,X-LC,12.50,277836.0,-68120.0,134834.0,-0.47,-19.69,48.53,19.28,1.0,-0.51,2.00,2.71,X40,ATH,IT
41,ITC,452.00,-39.19,47.0,X-LC,2.68,196578.0,-3560.0,24454.0,0.53,-1.78,12.44,10.44,4.0,-0.15,1.41,3.69,X40,NTT,FMCG
28,HAVELLS,2069.16,-8.45,33.0,X-MC,1.89,227042.0,-20689.0,95744.0,-0.60,-8.35,42.17,30.30,92.0,-0.22,1.63,3.83,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,35.0,X-SC,38.51,52297.0,-12709.0,163637.0,0.21,-19.55,312.90,232.18,198.0,-0.08,0.38,1.99,XY24,NTT,MISC
55,RELAXO,1176.00,-39.59,43.0,X-SC,4.02,82344.0,-62816.0,138741.0,-0.90,-43.27,168.49,52.31,136.0,-0.45,0.59,10.22,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.44,40.0,X-SC,4.33,94935.0,-33735.0,79033.0,1.79,-26.22,83.25,35.20,219.0,-0.43,0.68,8.96,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.16,50.0,X-SC,1.00,107946.0,5942.0,37986.0,0.50,5.83,35.19,43.06,122.0,0.16,0.78,23.94,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.41,108135.0,-19803.0,66936.0,-0.30,-15.48,61.90,36.84,143.0,-0.30,0.78,10.70,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-30.29,44.0,X-LC,12.50,277836.0,-68120.0,134834.0,-0.47,-19.69,48.53,19.28,1.0,-0.51,2.00,2.71,X40,ATH,IT
39,INFY,2275.00,-19.60,46.0,X-LC,7.54,313195.0,307.0,171380.0,-0.27,0.10,54.72,54.87,3.0,0.00,2.25,6.75,X40,BTT,IT
41,ITC,452.00,-39.19,47.0,X-LC,2.68,196578.0,-3560.0,24454.0,0.53,-1.78,12.44,10.44,4.0,-0.15,1.41,3.69,X40,NTT,FMCG
82,VBL,671.64,-18.99,51.0,X-LC,10.62,291757.0,-24085.0,136747.0,2.68,-7.63,46.87,35.67,5.0,-0.18,2.10,5.17,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,50.0,H-LC,7.25,249648.0,-11971.0,131190.0,0.53,-4.58,52.55,45.57,7.0,-0.09,1.80,6.23,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6633.33,53.0,L-SC,6.13,73881.0,-19929.0,96548.0,0.34,-21.24,130.68,81.67,269.0,-0.21,0.53,43.07,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,53.0,L-SC,28.82,83309.0,-30240.0,70288.0,1.83,-26.63,84.37,35.27,268.0,-0.43,0.60,103.90,XR,NTT,HOTELS
48,MASFIN,398.61,-20.05,44.0,H-SC,9.48,90795.0,-7185.0,28791.0,0.51,-7.33,31.71,22.05,152.0,-0.25,0.65,31.98,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,108.87,61.0,H-SC,11.92,131733.0,-10746.0,30272.0,0.45,-7.54,22.98,13.70,163.0,-0.35,0.95,55.48,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,54.0,H-SC,10.30,145105.0,-33966.0,99702.0,0.13,-18.97,68.71,36.71,135.0,-0.34,1.04,43.97,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,17.60,149256.0,36.0,144151.0,0.52,0.02,96.58,96.62,208.0,0.00,1.07,61.53,XY25,NTT,FINANCE
4,ANGELONE,3033.00,16.84,71.0,X-SC,9.31,206451.0,15445.0,42261.0,2.96,8.09,20.47,30.21,157.0,0.37,1.49,30.66,X40N,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,10.30,145105.0,-33966.0,99702.0,0.13,-18.97,68.71,36.71,135.0,-0.34,1.04,43.97,XR,NTT,BANKS
63,SHALBY,327.00,1195.06,57.0,M-SC,23.05,176161.0,-4982.0,50453.0,3.82,-2.75,28.64,25.10,235.0,-0.10,1.27,41.18,XY24,NTT,HEALTHCARE
31,HINDZINC,730.22,36.19,67.0,M-LC,9.50,222002.0,16926.0,95638.0,-0.56,8.25,43.08,54.89,52.0,0.18,1.60,34.34,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.50
1,25,43.96
2,50,75.82


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.45
LC    33.70
MC    21.81
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.63
X40      14.73
XY25     12.15
XR       10.97
X40N     10.26
AR        8.19
OX40N     7.65
X200      1.83
X5K       1.60
SR        1.13
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.53
X-LC    22.57
X-MC    16.06
M-SC    12.75
M-LC     5.35
H-LC     4.75
X-SC     4.70
H-MC     3.82
M-MC     1.63
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.06,-4.89,39.45
IT,12.94,-16.83,80.03
FINANCE,12.27,-10.79,61.01
MISC,6.90,-19.90,86.34
BANKS,6.36,-12.66,71.93
PAINTS,5.55,-19.57,37.39
ELECTRICAL,5.33,-11.13,50.12
INSURANCE,3.93,-1.24,39.09
AUTO,2.85,-44.63,113.77


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3313721.0,22
XR,1383179.0,14
AR,1189914.0,9
X40,788528.0,10
X40N,602498.0,8
OX40N,583451.0,10
XY25,493901.0,8
SR,263932.0,2
X5K,95638.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3449329.0,25
M-SC,1268115.0,15
X-LC,1099878.0,13
X-MC,1092237.0,11
X-SC,528594.0,6
H-MC,349724.0,3
H-LC,279938.0,3
M-LC,265330.0,4
L-SC,256164.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1197235.0      6
           XR         824786.0      7
           AR         820670.0      5
M-SC       XY24       686003.0      6
X-MC       XY24       588827.0      4
X-LC       X40        493544.0      5
           XY24       305619.0      2
X-SC       X40N       285924.0      4
H-SC       OX40N      272128.0      4
           SR         263932.0      2
X-MC       X40        215951.0      4
H-LC       AR         208665.0      2
X-LC       X40N       199617.0      3
H-MC       XY24       179244.0      1
X-MC       XY25       170502.0      2
L-SC       XR         166836.0      2
M-MC       XY24       165499.0      1
X-SC       XY24       163637.0      1
M-SC       AR         160579.0      2
           OX40N      150597.0      4
           XY25       144151.0      1
           XR         126785.0      2
X-MC       X40N       116957.0      1
M-LC       XR         105271.0      1
X-LC       XY25       101098.0      3
H-MC       XR          99082.0      1
M-LC       X5K         95638.0      1
L-SC       OX40N       89328.0      1
X-SC       X40         79033.0      1
H-MC       OX40N       71398.0      1
H-LC       X200        71273.0      1
H-SC       MH          70578.0      1
L-MC       XR          60419.0      1
L-LC       XY25        41386.0      1
M-LC       XY25        36764.0      1
           XY24        27657.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
